## NOTEBOOK DESCRIPTION:

This notebook checks the behaviour of the embedding space: 
- It first checks manually the nearest neighbors of some selected channels
- It also compute the user jumper score which is a first barrier on the evaluation of the embedding

In [2]:
import pickle
import os
import sys
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE

scriptpath = "/home/jouven/youtube_projects/"
sys.path.append(os.path.abspath(scriptpath))

from helpers.helpers_channels_more_300 import *
from helpers.helpers_channel_embedding import *

### Useful functions

In [3]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [4]:
PATH = '/dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/channel_sampling_then_combination/CONTEXT_True_100_SUBSAMPLING_FALSE_0.001_LR_0.005//models/embedding.csv.gz'

In [5]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/df_channels_en.tsv.gz", sep='\t')
# Select the rows being in the select channels
channelcrawler = channelcrawler[channelcrawler['channel'].apply(lambda row: row in channels_id)]

In [6]:
channelcrawler.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [7]:
channelcrawler['category_cc'].unique()

array(['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports',
       'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism',
       'People & Blogs', 'News & Politics', 'Science & Technology',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan],
      dtype=object)

### Manually check the neirest neighbors of some selected channels

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we have a sense of how good/bad is the embedding.

In [12]:
# Looking at the neirest neigbors of `Shaaanxo` being a girl fashion channel
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCMpOz2KEfkSdd5JeIJh_fxw,Howto & Style,2009-01-18,Shaaanxo,3215340,1422,3566.0,2.2780
1,UCCvoAe__WFYMNAEN-C-CtYA,Howto & Style,2008-08-05,Wayne Goss,3670000,1357,2740.0,2.2180
2,UCE1BqH-72EBs-siTbDAjOGQ,Howto & Style,2008-04-20,Marlena Stell,1520000,754,9894.0,2.6430
3,UCo5zIpjl2OQkYatd8R0bDaw,Howto & Style,2008-06-15,Dulce Candy,2220000,1055,6090.0,2.4605
4,UCjHyFOH80C4x_2n_Korx4QQ,People & Blogs,2010-11-03,ThatsHeart,2200000,432,6234.0,2.4680
5,UC8v4vz_n2rys6Yxpj8LuOBA,Howto & Style,2013-01-26,KathleenLights,4140000,979,2308.0,2.1680
6,UCzTKskwIc_-a0cGvCXA848Q,Howto & Style,2008-06-23,NikkieTutorials,12269235,742,312.0,2.0870
7,UCxj0QizmFhx7kVKSArHBCTA,Howto & Style,2011-03-09,Casey Holmes,1770000,730,8208.0,2.5370
8,UC21yq4sq8uxTcfgIxxyE9VQ,Howto & Style,2011-06-27,Carli Bybel,6190000,546,1234.0,2.0870
9,UCM3P_G21gOSVdepXrEFojIg,Howto & Style,2011-01-26,Rachel Talbott,1210000,337,13160.0,2.7555


In [8]:
# Looking at the neirest neigbors of `Alex Costa` being a men fashion channel
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCZyCposXwcyopaACep44maQ'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCZyCposXwcyopaACep44maQ,Howto & Style,2011-09-29,Alex Costa,2120000,543,5370.0,2.4170
1,UC1KbedtKa3d5dleFR6OjQMg,Howto & Style,2008-07-06,alpha m.,5360000,1116,1376.0,2.0870
2,UChNN7VBxPTiNrqjUaQd9bxA,Howto & Style,2012-09-14,Teachingmensfashion,4210000,1160,2093.0,2.1225
3,UCklSaeqb0wONcyEv44PV-iQ,Comedy,2017-08-08,The Daily Dropout,1907121,280,6761.0,2.4865
4,UCgFpJKyIywUc8arDxhABGPw,Science & Technology,2011-11-25,UnlockRiver.com,1411980,245,10323.0,2.6605
5,UCWsEZ9v1KC8b5VYjYbEewJA,Science & Technology,2011-03-22,Nick Ackerman,490000,1616,33875.0,3.2070
6,UCe0TLA0EsQbE-MjuHXevj2A,Howto & Style,2006-12-16,ATHLEAN-X™,8520000,1049,657.0,2.0870
7,UCu4X846OSea5YU6S8fIpy1A,Comedy,2012-06-20,BigDawsTv,6340000,225,1020.0,2.0870
8,UCSrnmu3W6YXWU_85DKT5arg,Comedy,2014-06-17,ThatWasEpic,3540000,189,2103.0,2.1255
9,UC5fqfItld8kQufe3djD6wAw,Howto & Style,2011-10-10,Brett Maverick,636000,222,25157.0,2.9560


In [9]:
# Looking at the neirest neigbors of `Magpiepony` which is a kids film
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,Magpiepony,736000,304,21120.0,2.9225
1,UC1AEadAUKi6Zt-G3PatrU-Q,Film and Animation,2012-08-19,Letupita725HD★,2698245,2115,4054.0,2.3350
2,UCcZjYWpM72s-1cinYfeSCag,People & Blogs,2010-08-05,DisneyFanatic2364,205000,1026,88028.0,4.3405
3,UCD_VOth7RmckN6DbmFJa__A,Entertainment,2013-08-31,The Brony Notion,256132,158,70689.0,3.7420
4,UC9qgVkKRZUXcgHdf35Z-8dw,Film and Animation,2013-06-24,My Little Pony Offic...,874016,1840,12248.0,2.6960
5,UCkQ3eqNNaFrBZ_WbnCIMN8w,Film and Animation,2011-10-19,shgurr,2060000,68,6256.0,2.4685
6,UC1RMBtYn0Fw8Pf7KuvkrRNw,Gaming,2009-11-01,Vannamelon,919000,241,17845.0,2.8605
7,UCcGuE4cmWhB4XLjSKA8eRFA,Entertainment,2014-08-30,WatchPony,117000,341,144609.0,4.0540
8,UCPflSzRRyzK2hhbVxNAnrhw,Film and Animation,2012-03-31,Pinkie Pie,1229394,451,11840.0,2.6965
9,UCZBY6V8Lxmwu8gGRBOyO11w,Gaming,2014-06-16,Kubz Scouts,3160000,1440,3303.0,2.2885


In [10]:
# Looking at neirest neigbors of `Beyond the trailer` a channel for analysis of films
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCAXR2zenCwvRIyQd9ydtfaA'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCAXR2zenCwvRIyQd9ydtfaA,Entertainment,2008-11-03,Beyond The Trailer,853000,5064,19744.0,2.8635
1,UCWvMmm_sSdgALpo1Ci4WvtQ,Entertainment,2007-02-04,Collider Videos,583000,9074,30710.0,3.0665
2,UCQMbqH7xJu5aTAPQ9y_U7WQ,Entertainment,2009-03-05,Fandom Entertainment...,1570000,10813,9597.0,2.5995
3,UCzNWVDZQ55bjq8uILZ7_wyQ,Film and Animation,2011-06-27,FRESH Movie Trailers...,4150000,6543,2211.0,2.1430
4,UCRgUg0jJUgGMadGPzzmu8cw,Entertainment,2008-07-16,GameSpot Universe,596000,4512,28150.0,3.0100
5,UCi8e0iOVk1fEOogdfu4YgfA,Film and Animation,2011-04-01,Movieclips Trailers,13900000,8084,267.0,2.0870
6,UCCqEeDAUf4Mg0GgEN658tkA,Entertainment,2011-01-06,Chris Stuckmann,1580000,1400,8486.0,2.5600
7,UC7v3-2K1N84V67IF-WTRG-Q,Entertainment,2007-03-07,Jeremy Jahns,1600000,1584,8351.0,2.5415
8,UCTAgbu2l6_rBKdbTvEodEDw,Entertainment,2007-06-17,Nerdist,2690000,2370,4427.0,2.3700
9,UCP1iRaFlS5EYjJBryFV9JPw,Entertainment,2015-11-25,Looper,4820000,2533,1758.0,2.0870


In [15]:
nearest_neighbors_id = get_k_nearest_neighbors(PATH, dict_channel_ind['UCaCTtPkowVggMiUYhQmlcMw'], dict_ind_channel, k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel']).merge(channelcrawler)
nearest_neighbors

,channel,category_cc,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,UCaCTtPkowVggMiUYhQmlcMw,Sports,2015-12-30,CrossFit Invictus,27200,196,440262.0,7.1160
1,UCg4o4z4Xrk9SG4wX3Kpmydw,Sports,2011-04-21,Noah Ohlsen,50200,201,258307.0,6.4685
2,UCWDto2R36n9pwwXq1HgjYHA,Sports,2015-07-09,Julie Foucher,16100,127,690819.0,10.0600
3,UCj6Pm9XJJBdxFrJ4nw6ZEAQ,Education,2015-08-02,Working Against Grav...,16900,128,639884.0,9.8300
4,UCTw6CZg_va2CWaENxhhPcSA,Sports,2013-07-30,USN UK - Ultimate Sp...,19200,226,605238.0,9.5685
5,UCQHhd5lxVEkGidVXXegTisw,Sports,2014-07-01,Structure Your Stren...,19300,485,541864.0,8.7070
6,UCrfUPwsgeu_Gp9xJ1Z9v1Jg,Sports,2011-01-14,Gracie Davis,18700,1147,604357.0,9.6795
7,UCzUd3H_e_qWqs4dR4GxVHOQ,People & Blogs,2017-02-14,Training Think Tank,19200,287,582830.0,6.6245
8,UCoQ9JsZw8RT8w6uqDmq12Hg,Sports,2017-03-20,Logan Sneed,30800,593,418137.0,6.1990
9,UCs9b1SWEVJ2N4_l6eJIHJ2Q,Sports,2015-04-03,Ben Bergeron,30600,128,396788.0,6.9700


### User jumper implementation

In [ ]:
# We randomly choose 3 000 users and for each user we select a random pair of channel
# channels_tuple = [(channel_1, channel_2), (...)]
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_user_walk.pkl",'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

In [20]:
files = [PATH]
users_walk_tab, users_walk_tab_new, ranking_position_tab = get_user_walk_and_position_ratio(files, channels_tuple)

In [22]:
print('User walk = ' + str(users_walk_tab))
print("User walk (new way) = " + str(users_walk_tab_new))
print('Ranking position ratio = ' + str(ranking_position_tab))

User walk = [0.5132652493392449]
User walk (new way) = [1.701289078870937]
Ranking position ratio = [0.0760710714726577]


### Plot the embedding

In [8]:
ix_to_word, word_to_ix = channel_to_name(channelcrawler, dict_channel_ind)

In [ ]:
# Project the embedding in 2D

EMBEDDING = get_dataframe_in_embedding_space(PATH)

print("\n', 'running TSNE...")
tsne = TSNE(n_components = 2).fit_transform(EMBEDDING)
print("tsne.shape: ", tsne.shape) 

# Save the projected embedding in the 2D plane
np.save('/home/jouven/youtube_projects/word2vec_pytorch/embedding_space/run_channels_more_300/channel_sampling_then_combination/CONTEXT_True_100_SUBSAMPLING_False_LR_0.005/tsne.npy', tsne)



', 'running TSNE...


Plot to visualize the embeddingg in 2D.
The channels appearing in the plot are the channels having more than 4000k subsribers. In addition, we have 20 colored channels representing the selected pairs in the "axis projection process". 

In [ ]:
############ VISUALIZING ############
SAMPLES = 500
x, y = [], []
annotations = []
for idx, coord in enumerate(tsne):
    # print(coord)
    annotations.append(ix_to_word[idx])
    x.append(coord[0])
    y.append(coord[1])

test_words = [dict_ind_channel[channel] for channel in random.sample(range(len(channels_id)), SAMPLES)]
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_more_4000k_subs.pkl",'rb') as f:
     test_words = pickle.load(f)
f.close()
test_words = random.sample(test_words, SAMPLES)
plt.figure(figsize = (50, 50))
for i in range(len(test_words)):
    word = ix_to_word[dict_channel_ind[test_words[i]]]
    vocab_idx = word_to_ix[word]
    plt.scatter(x[vocab_idx], y[vocab_idx])
    
    plt.annotate(word, xy = (x[vocab_idx], y[vocab_idx]), \
            ha='right',va='bottom', color = 'gray', fontsize = 12)
#plt.savefig("/home/jouven/youtube_projects/w2v.png")
plt.title("Embedding visualization in 2D using channels having more than 4000k subsribers")
plt.show()